# Load kaggle api token

In [ ]:
from google.colab import files
files.upload()

# Install

In [11]:
!pip install git+https://github.com/cene555/ru-clip-tiny.git

  Cloning https://github.com/cene555/ru-clip-tiny.git to /tmp/pip-req-build-g90bca5i
  Running command git clone -q https://github.com/cene555/ru-clip-tiny.git /tmp/pip-req-build-g90bca5i
  Created wheel for rucliptiny: filename=rucliptiny-1.0-py3-none-any.whl size=6832 sha256=02f832e5c8fc7b4a84f3e4e681d77a7a5f772903687e94416172d9ae7b7ad3e5
  Stored in directory: /tmp/pip-ephem-wheel-cache-53351fxb/wheels/de/f8/47/2a2023df2bc2fa5ea07e4c1313ce509c8e9755863aa290cd46
Successfully built rucliptiny


Download model

In [ ]:
!gdown -O ru-clip-tiny.pkl https://drive.google.com/uc?id=1-3g3J90pZmHo9jbBzsEmr7ei5zm3VXOL

Downloading...
From: https://drive.google.com/uc?id=1-3g3J90pZmHo9jbBzsEmr7ei5zm3VXOL
To: /content/ru-clip-tiny.pkl
100% 153M/153M [00:00<00:00, 226MB/s]


Download dataset

In [ ]:
!gdown https://drive.google.com/uc?id=1VRDYOvzvASd8PHSBTbp6En47-I5X88YU

Downloading...
From: https://drive.google.com/uc?id=1VRDYOvzvASd8PHSBTbp6En47-I5X88YU
To: /content/ru_flickr.csv
100% 5.73M/5.73M [00:00<00:00, 129MB/s]


In [ ]:
!pip install -q kaggle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download hsankesara/flickr-image-dataset

100% 8.15G/8.16G [01:14<00:00, 97.6MB/s]
100% 8.16G/8.16G [01:14<00:00, 118MB/s] 


In [ ]:
!unzip /content/flickr-image-dataset.zip

Выходные данные были обрезаны до нескольких последних строк (5000).
  inflating: flickr30k_images/flickr30k_images/flickr30k_images/535529555.jpg  
  inflating: flickr30k_images/flickr30k_images/flickr30k_images/5356236664.jpg  
  inflating: flickr30k_images/flickr30k_images/flickr30k_images/5356318668.jpg  
  inflating: flickr30k_images/flickr30k_images/flickr30k_images/535830521.jpg  
  inflating: flickr30k_images/flickr30k_images/flickr30k_images/5359437945.jpg  
  inflating: flickr30k_images/flickr30k_images/flickr30k_images/5361355399.jpg  
  inflating: flickr30k_images/flickr30k_images/flickr30k_images/53614287.jpg  
  inflating: flickr30k_images/flickr30k_images/flickr30k_images/536147209.jpg  
  inflating: flickr30k_images/flickr30k_images/flickr30k_images/5361551308.jpg  
  inflating: flickr30k_images/flickr30k_images/flickr30k_images/536156200.jpg  
  inflating: flickr30k_images/flickr30k_images/flickr30k_images/5362687336.jpg  
  inflating: flickr30k_images/flickr30k_images/

# Import

In [1]:
from rucliptiny import RuCLIPtiny
from rucliptiny.trainer import Trainer

import torch
import transformers
from PIL import Image
import transformers
from transformers import BertTokenizer
from torchvision import transforms
import pandas as pd
import cv2
from sklearn.model_selection import train_test_split

# Load data

In [2]:
df = pd.read_csv('ru_flickr.csv')

In [3]:
df.head()

,image_name,text
0,1000092795.jpg,Два молодых парня с шаткими волосами смотрят н...
1,10002456.jpg,Несколько человек в жестких люках управляют ги...
2,1000268201.jpg,Ребенок в розовом платье поднимается по лестни...
3,1000344755.jpg,Кто-то в синей рубашке и шляпе стоит на полу и...
4,1000366164.jpg,"Двое мужчин, один в темной рубашке, другой в ч..."


In [4]:
train_df, val_df = train_test_split(df, test_size=0.1, random_state=655)

In [5]:
train_df.to_csv('train.csv', index=False)
val_df.to_csv('val.csv', index=False)


# Load model

In [6]:
device = 'cuda'

In [7]:
model = RuCLIPtiny().to(device)
model.load_state_dict(torch.load('/content/ru-clip-tiny.pkl'))

<All keys matched successfully>

# Train model

new train batch size = train_batch_size * grad_accum

In [8]:
trainer = Trainer(train_dataframe='/content/train.csv', train_dir='/content/flickr30k_images/flickr30k_images',
                  val_dataframe='/content/val.csv', val_dir='/content/flickr30k_images/flickr30k_images',
                  train_batch_size=64, grad_accum=16)

train batch size = 1024


In [9]:
model = trainer.train_model(model, epochs_num=2, device=device, verbose=2)

start training epoch 0


  0%|          | 0/447 [00:00<?, ?it/s]

15/447 total_loss 4.162946701049805
47/447 total_loss 4.0285964012146
79/447 total_loss 3.902125597000122
111/447 total_loss 3.9306960105895996
143/447 total_loss 3.7845730781555176
175/447 total_loss 3.748833656311035
207/447 total_loss 3.8492789268493652
239/447 total_loss 3.796928882598877
271/447 total_loss 3.8242063522338867
303/447 total_loss 3.8062543869018555
335/447 total_loss 3.749683380126953
367/447 total_loss 3.6933586597442627
399/447 total_loss 3.633063316345215
431/447 total_loss 3.7762467861175537
start val epoch 0


  0%|          | 0/50 [00:00<?, ?it/s]

val loss = 1.2007425403594971
start training epoch 1


  0%|          | 0/447 [00:00<?, ?it/s]

15/447 total_loss 3.6580593585968018
47/447 total_loss 3.512634038925171
79/447 total_loss 3.5739312171936035
111/447 total_loss 3.5712878704071045
143/447 total_loss 3.6271822452545166
175/447 total_loss 3.401965856552124
207/447 total_loss 3.5194714069366455
239/447 total_loss 3.6230578422546387
271/447 total_loss 3.547368049621582
303/447 total_loss 3.481950283050537
335/447 total_loss 3.4620466232299805
367/447 total_loss 3.489931344985962
399/447 total_loss 3.521660804748535
431/447 total_loss 3.4608263969421387
start val epoch 1


  0%|          | 0/50 [00:00<?, ?it/s]

val loss = 1.13596888422966
